# Code for Implementing Locality-Sensitive Hashing for Finding Similar Netflix Users

In [6]:
import numpy as np
import sys
import time
from scipy.sparse import csr_matrix
from collections import defaultdict
from itertools import combinations, islice
from google.colab import drive
import hashlib

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/user_movie_rating.npy'

def main():
    try:
        # Prompting for random seed
        seed = int(input("Enter random seed: "))
        print(f"Random seed set to: {seed}")
        np.random.seed(seed)

        # Loading data
        print("Loading data...")
        start_time = time.time()
        data = np.load(data_path, mmap_mode='r')
        load_time = time.time() - start_time
        print(f"Data loaded in {load_time:.2f} seconds.")

        # Adjusting IDs to start from 0
        user_ids = data[:, 0].astype(np.int32) - 1
        movie_ids = data[:, 1].astype(np.int32) - 1
        n_users = user_ids.max() + 1
        n_movies = movie_ids.max() + 1
        print(f"Number of users: {n_users}")
        print(f"Number of movies: {n_movies}")

        # Values for user-movie matrix
        values = np.ones(len(user_ids), dtype=np.int8)
        user_movie_matrix = csr_matrix((values, (user_ids, movie_ids)), shape=(n_users, n_movies))

        print("\nUsing Random Permutations for MinHash signatures.")

        # Signature length
        n = 100
        print(f"Using signature length: {n}")

        # Generating random permutations
        print("Generating random permutations...")
        start_perm = time.time()
        permutations = np.empty((n, n_movies), dtype=np.uint32)
        for i in range(n):
            if (i+1) % 10 == 0 or i == 0:
                print(f"Generating permutation {i+1}/{n}")
            permutations[i] = np.random.permutation(n_movies).astype(np.uint32)
        perm_time = time.time() - start_perm
        print(f"Random permutations generated in {perm_time:.2f} seconds.")

        # Computing ranks - for each hash function, the rank of each movie
        print("Computing ranks...")
        start_rank = time.time()
        ranks = np.argsort(permutations, axis=1)
        rank_time = time.time() - start_rank
        print(f"Computed ranks in {rank_time:.2f} seconds.")

        # Preparing user_movies for signature computation
        print("Preparing user movies for signature computation...")
        # Finding the maximum number of movies rated by any user
        user_movie_counts = user_movie_matrix.indptr[1:] - user_movie_matrix.indptr[:-1]
        max_movies = user_movie_counts.max()
        # Initializing a 2D array: first column is the count, rest are movie indices
        user_movies = np.zeros((n_users, max_movies + 1), dtype=np.uint32)
        for user_id in range(n_users):
            start_idx = user_movie_matrix.indptr[user_id]
            end_idx = user_movie_matrix.indptr[user_id + 1]
            num_movies = end_idx - start_idx
            user_movies[user_id, 0] = num_movies
            if num_movies > 0:
                user_movies[user_id, 1:num_movies + 1] = user_movie_matrix.indices[start_idx:end_idx]
        print("User movies prepared.")

        # Computing MinHash signatures
        print("Computing MinHash signatures...")
        start_time = time.time()
        signatures = np.full((n_users, n), n_movies, dtype=np.uint32)  # Initializing with max rank

        for i in range(n):
            if (i+1) % 10 == 0 or i == 0:
                print(f"Processing hash function {i+1}/{n}")
            # For each user, finding the minimum rank for hash function i
            for user_id in range(n_users):
                num_movies = user_movies[user_id, 0]
                if num_movies == 0:
                    continue
                user_rated_movies = user_movies[user_id, 1:num_movies + 1]
                # Finding the minimum rank among the user's rated movies for this hash function
                min_rank = ranks[i, user_rated_movies].min()
                signatures[user_id, i] = min_rank
        sig_time = time.time() - start_time
        print(f"Computed MinHash signatures in {sig_time:.2f} seconds.")

        # Choosing banding parameters
        b_bands = 20
        r_rows = 5
        print(f"Using {b_bands} bands and {r_rows} rows per band.")

        print("Starting LSH banding...")
        start_time = time.time()
        candidate_pairs_list = []  # Initializing as a list instead of a set

        max_bucket_size = 2000
        print(f"Using maximum bucket size: {max_bucket_size}")

        for band in range(b_bands):
            print(f"\nProcessing band {band + 1}/{b_bands}")
            band_start_time = time.time()
            start = band * r_rows
            end = start + r_rows
            band_signatures = signatures[:, start:end]

            # Converting band_signatures to bytes for hashing
            # Each row in band_signatures is a sub-signature
            band_hashes = np.array([
                hashlib.md5(band_signatures[user_id].tobytes()).hexdigest()
                for user_id in range(n_users)
            ])

            # Organizing users into buckets
            buckets = defaultdict(list)
            for user_id, bh in enumerate(band_hashes):
                buckets[bh].append(user_id)

            # Collecting candidate pairs
            for bucket_id, bucket_users in buckets.items():
                bucket_size = len(bucket_users)
                if 1 < bucket_size < max_bucket_size:
                    # Generating all possible pairs within the bucket
                    for u1, u2 in combinations(bucket_users, 2):
                        pair = (min(u1, u2), max(u1, u2))
                        candidate_pairs_list.append(pair)  # Appending to list instead of adding to set
                elif bucket_size >= max_bucket_size:
                    print(f"Skipping bucket with size {bucket_size}.")
            band_time = time.time() - band_start_time
            print(f"Processed band {band + 1} in {band_time:.2f} seconds.")

        # Converting the list of candidate pairs to a NumPy array
        print("\nConverting candidate pairs to NumPy array and removing duplicates...")
        candidate_pairs_array = np.array(candidate_pairs_list, dtype=np.uint32)
        del candidate_pairs_list  # Free memory by deleting the list

        # Removing duplicate pairs using NumPy's unique function
        candidate_pairs_unique = np.unique(candidate_pairs_array, axis=0)
        print(f"Number of unique candidate pairs: {candidate_pairs_unique.shape[0]}")

        lsh_time = time.time() - start_time
        print(f"\nCompleted LSH banding in {lsh_time:.2f} seconds.")
        print(f"Number of candidate pairs: {candidate_pairs_unique.shape[0]}")

        # Verifying candidate pairs with estimated similarity
        print("\nVerifying candidate pairs using estimated similarity...")
        start_time = time.time()
        similar_pairs = []
        threshold = 0.5

        n_signatures = signatures.shape[1]  # n

        # Defining a generator to process candidate pairs in batches
        def batch_iterator(arr, batch_size=100000):
            for i in range(0, arr.shape[0], batch_size):
                yield arr[i:i+batch_size]

        for batch_idx, batch in enumerate(batch_iterator(candidate_pairs_unique, batch_size=100000)):
            print(f"Processing batch {batch_idx + 1}")
            # Extracting u1 and u2 from the batch
            u1_batch = batch[:, 0]
            u2_batch = batch[:, 1]

            # Estimating similarity by comparing signatures
            matches = (signatures[u1_batch] == signatures[u2_batch])
            est_similarity = matches.sum(axis=1) / n_signatures

            # Filtering pairs based on estimated similarity
            valid_indices = np.where(est_similarity >= 0.4)[0]
            if valid_indices.size == 0:
                continue  # No pairs pass the estimated similarity threshold in this batch

            # Extracting valid pairs
            valid_u1 = u1_batch[valid_indices]
            valid_u2 = u2_batch[valid_indices]

            # Computing exact Jaccard similarity using sorted movie indices
            for i in range(valid_u1.size):
                u1 = valid_u1[i]
                u2 = valid_u2[i]
                num_u1 = user_movies[u1, 0]
                num_u2 = user_movies[u2, 0]
                if num_u1 == 0 or num_u2 == 0:
                    continue
                movies_u1 = user_movies[u1, 1:num_u1 + 1]
                movies_u2 = user_movies[u2, 1:num_u2 + 1]
                # Computing intersection and union using NumPy's intersect1d
                intersection_size = np.intersect1d(movies_u1, movies_u2, assume_unique=True).size
                union_size = num_u1 + num_u2 - intersection_size
                similarity = intersection_size / union_size if union_size > 0 else 0.0
                if similarity > threshold:
                    similar_pairs.append((u1 + 1, u2 + 1))  # Adding 1 to match original IDs

        verify_time = time.time() - start_time
        print(f"Verified candidate pairs in {verify_time:.2f} seconds.")
        print(f"Number of similar pairs with Jaccard similarity > {threshold}: {len(similar_pairs)}")

        # Writing similar pairs to result file
        print("\nWriting similar pairs to result.txt...")
        with open('result.txt', 'w') as f:
            for u1, u2 in similar_pairs:
                f.write(f'{u1},{u2}\n')
        print("Completed writing similar pairs to result.txt.")

        # Summary of results
        total_time = load_time + perm_time + rank_time + sig_time + lsh_time + verify_time
        print("\n=== Summary of Results ===")
        print(f"Random seed: {seed}")
        print(f"Signature length (n): {n}")
        print(f"Number of bands (b): {b_bands}")
        print(f"Rows per band (r): {r_rows}")
        print(f"Maximum bucket size processed: {max_bucket_size}")
        print(f"Number of candidate pairs: {candidate_pairs_unique.shape[0]}")
        print(f"Number of similar pairs found: {len(similar_pairs)}")
        print(f"Total computation time: {total_time:.2f} seconds.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter random seed: 359
Random seed set to: 359
Loading data...
Data loaded in 0.19 seconds.
Number of users: 103703
Number of movies: 17770

Using Random Permutations for MinHash signatures.
Using signature length: 100
Generating random permutations...
Generating permutation 1/100
Generating permutation 10/100
Generating permutation 20/100
Generating permutation 30/100
Generating permutation 40/100
Generating permutation 50/100
Generating permutation 60/100
Generating permutation 70/100
Generating permutation 80/100
Generating permutation 90/100
Generating permutation 100/100
Random permutations generated in 0.08 seconds.
Computing ranks...
Computed ranks in 0.19 seconds.
Preparing user movies for signature computation...
User movies prepared.
Computing MinHash signatures...
Processing hash function 1/100
Processing hash function 10/100
Processing hash functi